<a href="https://colab.research.google.com/github/LesyaDeryabina/homework2/blob/main/homework2_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Работа с файлами, pandas + парсинг
Собираем корпус эко-новостей с сайта новостного издания "Ведомости"

[По этой ссылке можно найти главную страницу](https://www.vedomosti.ru/ecology)

Что нам предстоит сделать:

достать все заголовки новостей в главной страницы + текст каждой новости

сохранить в датафрейм с колонками "ссылка", "дата", "заголовок", "текст"

сохранить датафрейм в файл

положить код и получившийся файл в свой репозиторий на гитхабе

# Шаг 1: Достаем ссылки на каждый релиз

In [55]:
import requests as rq
from bs4 import BeautifulSoup
import json
import pandas as pd

In [ ]:
url = "https://www.vedomosti.ru/ecology"
main_page = rq.get(url)
print(main_page)

In [ ]:
soup = BeautifulSoup(main_page.text, features="html.parser")
print(soup.prettify())

In [ ]:
urls = []

for link in soup.find_all('a'):
    urls.append(link.get('href'))

urls

In [ ]:
new_urls = []
for link in soup.find_all('a'):
  if 'release' in link.get('href') and link.get('href') not in new_urls:
    new_urls.append(link.get('href')) 

new_urls

In [ ]:
full_urls = []
for i in new_urls:
  full_urls.append(f'https://www.vedomosti.ru{i}')

full_urls

# Шаг 2: Достаем все ссылки на статьи из каждого релиза

In [61]:
for link in full_urls:
  url0 = full_urls[0]
  page0 = rq.get(url0)
  soup0 = BeautifulSoup(page0.text, features="html.parser")

In [62]:
articles0 = []

for link in soup0.find_all('a', href=True):
  if 'articles' in link.get('href') or 'columns' in link.get('href') and link.get('href') not in articles0:
      articles0.append(link.get('href'))

articles0

['/ecology/climate/articles/2022/12/12/954783-metan-nedolgovechnii-no-kovarnii',
 '/ecology/climate/columns/2022/12/12/954773-klimaticheskie-izmeneniya-i-ustoichivoe-razvitie',
 '/ecology/science_and_technology/articles/2022/12/12/954778-dobivayuschei-otrasli-prognoziruyut-diversifikatsiyu']

In [63]:
def AL(url0):
  page0 = rq.get(url0)
  soup0 = BeautifulSoup(page0.text, features="html.parser")
  articles0 = []
  for link in soup0.find_all('a', href=True):
    if 'articles' in link.get('href') or 'columns' in link.get('href') and link.get('href') not in articles0:
      articles0.append(link.get('href'))  
  return articles0

In [ ]:
all_articles = []
for link in full_urls:
  articles = AL(link)
  all_articles.extend(articles)

all_articles

In [ ]:
full_artricles = []

for i in all_articles:
  full_artricles.append(f'https://www.vedomosti.ru{i}')

full_artricles

# Шаг 3: Проходимся по каждой ссылке и достаем тексты статей

In [66]:
for link in full_artricles:
  url_0 = full_artricles[0]
  page_0 = rq.get(url_0)
  soup_0 = BeautifulSoup(page_0.text, features="html.parser")

In [ ]:
script = soup_0.find_all('script')[1]
script

In [ ]:
data = json.loads(script.get_text())
data

In [ ]:
title = data["name"]
date = data["datePublished"][:10]
print(title, date)

In [ ]:
text_list = soup_0.find_all('p', {'class' : 'box-paragraph__text'})
text_list

In [ ]:
text = []
for i in text_list:
  text.append(i.text)
text

In [ ]:
final_text = ' '.join(text)
final_text

In [73]:
def News(url_0):
  page_0 = rq.get(url_0)
  soup_0 = BeautifulSoup(page_0.text, features="html.parser")
  script = soup_0.find_all('script')[1]
  data = json.loads(script.get_text())
  title = data["name"]
  date = data["datePublished"][:10]
  text_list = soup_0.find_all('p', {'class' : 'box-paragraph__text'})
  text = []
  for i in text_list:
    text.append(i.text)
  final_text = ' '.join(text)
  return url_0, title, date, final_text

In [74]:
all_news = []

for link in full_artricles:
  new = News(link)
  all_news.append(new)

In [75]:
df = pd.DataFrame(all_news)
df.head()

,0,1,2,3
0,https://www.vedomosti.ru/ecology/climate/artic...,"Метан – недолговечный, но коварный",2022-12-12,Российские нефтегазовые компании активно участ...
1,https://www.vedomosti.ru/ecology/climate/colum...,Климатические изменения и устойчивое развитие,2022-12-12,Современные климатические изменения представля...
2,https://www.vedomosti.ru/ecology/science_and_t...,Добывающей отрасли прогнозируют диверсификацию,2022-12-12,В условиях макроэкономической нестабильности и...
3,https://www.vedomosti.ru/ecology/climate/artic...,Энергетика с нулевым следом,2022-12-08,"Климатическая повестка, декарбонизация, зелена..."
4,https://www.vedomosti.ru/ecology/science_and_t...,«Россия» станет самым мощным ледоколом в мире,2022-12-08,"И. о. генерального директора ФГУП «Атомфлот», ..."


In [76]:
df.columns = ['link', 'title', 'date', 'text']

In [79]:
df.head()

,link,title,date,text
0,https://www.vedomosti.ru/ecology/climate/artic...,"Метан – недолговечный, но коварный",2022-12-12,Российские нефтегазовые компании активно участ...
1,https://www.vedomosti.ru/ecology/climate/colum...,Климатические изменения и устойчивое развитие,2022-12-12,Современные климатические изменения представля...
2,https://www.vedomosti.ru/ecology/science_and_t...,Добывающей отрасли прогнозируют диверсификацию,2022-12-12,В условиях макроэкономической нестабильности и...
3,https://www.vedomosti.ru/ecology/climate/artic...,Энергетика с нулевым следом,2022-12-08,"Климатическая повестка, декарбонизация, зелена..."
4,https://www.vedomosti.ru/ecology/science_and_t...,«Россия» станет самым мощным ледоколом в мире,2022-12-08,"И. о. генерального директора ФГУП «Атомфлот», ..."


In [78]:
df.to_excel('ecology_news.xlsx')